# 01/20

%matplotlib inline

In [1]:
description = 'mobilenet_from_pretrain_emotion_freeze_base'

In [2]:

import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import math

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize



In [3]:
import tqdm
import json

In [4]:
train_df = pd.read_csv('../../data/Manually_Annotated_file_lists/training_face_mesh_crop.csv')
train_df.subDirectory_filePath = '../../data/Manually_Annotated_Images_FaceMesh_Cropped/' + train_df.subDirectory_filePath

#train_df_2 = pd.read_csv('../../data/Automatically_annotated_file_list/automatically_annotated_face_mesh_crop.csv')
#train_df_2.subDirectory_filePath = '../../data/Automatically_Annotated_Images_FaceMesh_Cropped/' + train_df_2.subDirectory_filePath
#train_df = train_df.append(train_df_2)
#del train_df_2

train_df = train_df[train_df['have_facemesh']]

In [5]:
from eason_utils import DataFrameBatchIterator
from eason_utils import lprint, now_time_string, log_file_name, change_log_file_name

change_log_file_name(f'''{log_file_name.replace('.log', '')}_{description}.log''')

log_file_name='01_20_22:19:49.log'


In [6]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(allow_growth = True))
sess = tf.compat.v1.Session(config = config)

In [13]:
mobilenet_pretrained = tf.keras.models.load_model('../models/affectnet_emotions/mobilenet_7.h5')

In [15]:
mobilenet_output = mobilenet_pretrained.get_layer("global_pooling").output

In [16]:
mobilenet_pretrained.trainable = False

In [18]:
hidden_layers = [256]

In [19]:
"""valaence_feat = mobilenet_output
for size in hidden_layers:
    valence_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_valence')(valaence_feat)
outputs_valence = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_valence')(valence_feat)

arousal_feat = mobilenet_output
for size in hidden_layers:
    arousal_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_arousal')(arousal_feat)
outputs_arousal = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_arousal')(arousal_feat)
"""
emotion_feat = mobilenet_output
for size in hidden_layers:
    emotion_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_emotion')(emotion_feat)
emotions_count = len(np.unique(train_df.expression))
outputs_emotion = tf.keras.layers.Dense(emotions_count, activation = 'softmax', name = 'outputs_emotion')(emotion_feat)

In [20]:
model = tf.keras.Model(inputs=mobilenet_pretrained.input,
                       outputs=(outputs_emotion) , name="mobilenet_train")

In [21]:
model.summary()

Model: "mobilenet_train"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_pad (ZeroPadding2D)   (None, 225, 225, 3)       0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                   

In [22]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()
# Instantiate a loss function.
MSE_loss = tf.keras.losses.MeanSquaredError()
SCC_loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [23]:
batch_size = 32
epochs = 16

logs = []

In [ ]:
from telegram_notifier import send_message
import time
start_time = time.time()


for epoch in range(epochs):
    total_loss = 0
    total_step = 0
    total_emotion_correct = 0
    total_valence_loss = 0
    total_arousal_loss = 0
    total_emotion_loss = 0
    total_process = 0
    for step, row in enumerate(DataFrameBatchIterator(train_df, batch_size=batch_size)):

        imgs = row.subDirectory_filePath.apply(lambda x: cv2.resize(
            cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))
        img_array = np.array(list(imgs))

        y_valence = np.array(row.valence)
        y_arousal = np.array(row.arousal)
        y_emotion = np.array(row.expression)

        with tf.GradientTape() as tape:
            logits = model(img_array, training=True)

            pred_emotion = logits

            emotion_loss = SCC_loss(y_emotion, pred_emotion)

            loss = emotion_loss

            grads = tape.gradient(loss, model.trainable_weights)

            optimizer.apply_gradients(zip(grads, model.trainable_weights))
        total_loss += float(loss)
        total_step += 1
        
        total_process += len(row)

        #valence_loss = float(valence_loss)
        #arousal_loss = float(arousal_loss)
        emotion_loss = float(emotion_loss)
        
        #total_valence_loss += valence_loss
        #total_arousal_loss += arousal_loss
        total_emotion_loss += emotion_loss
        
        emotion_correct = int(sum(pred_emotion.numpy().argmax(axis = 1) == y_emotion))
        total_emotion_correct += emotion_correct
        
        log = {
            'emotion_correct': emotion_correct,
            #'valence_loss': valence_loss,
            #'arousal_loss': arousal_loss,
            'emotion_loss': emotion_loss,
            'time_take': time.time() - start_time
        }
        log = json.dumps(log)
        lprint(log)
        """if step % 10 == 0:
            lprint(
                f"Training loss at epoch {epoch} step {step}: {total_loss/total_step}\n",
                f"This round's {valence_loss=}, {arousal_loss=}, {emotion_loss=}"
            )
            lprint("Seen so far: %s samples" % ((step + 1) * batch_size))
            lprint(f"--- {time.time() - start_time} seconds for iter {total_step} step, each step have {batch_size}, so the model train with {total_step*batch_size} img this iter ---\n")
            total_loss = 0
            total_step = 0

            start_time = time.time()"""
    save_model_path = f"models/{now_time_string}_{description}_epoch{epoch}_batch_{batch_size}"
    model.save(save_model_path)
    lprint(f"Save {save_model_path}")
    send_message(f"Save {save_model_path}\nepoch {epoch} is finish")
    log = {
        'model_path': save_model_path,
        'total_emotion_correct': total_emotion_correct,
        'total_loss': total_loss,
        #'total_valence_loss': total_valence_loss,
        #'total_arousal_loss': total_arousal_loss,
        'total_emotion_loss': total_emotion_loss,
        'total_process': total_process,
        'time_take': time.time() - start_time
    }
    log = json.dumps(log)
    lprint(log)

{"emotion_correct": 1, "emotion_loss": 2.6201400756835938, "time_take": 6.116149663925171}
{"emotion_correct": 8, "emotion_loss": 2.1831610202789307, "time_take": 6.726681232452393}
{"emotion_correct": 13, "emotion_loss": 1.858689785003662, "time_take": 7.120045185089111}
{"emotion_correct": 16, "emotion_loss": 1.61135995388031, "time_take": 7.639240503311157}
{"emotion_correct": 15, "emotion_loss": 1.4313007593154907, "time_take": 8.089582443237305}
{"emotion_correct": 16, "emotion_loss": 1.450534701347351, "time_take": 8.389005422592163}
{"emotion_correct": 15, "emotion_loss": 1.4991919994354248, "time_take": 8.821369886398315}
{"emotion_correct": 16, "emotion_loss": 1.380983829498291, "time_take": 9.365677118301392}
{"emotion_correct": 18, "emotion_loss": 1.4634310007095337, "time_take": 9.672704935073853}
{"emotion_correct": 16, "emotion_loss": 1.5362815856933594, "time_take": 9.868897676467896}
{"emotion_correct": 19, "emotion_loss": 0.9682810306549072, "time_take": 10.32232046127

{"emotion_correct": 21, "emotion_loss": 1.429732084274292, "time_take": 39.332250118255615}
{"emotion_correct": 22, "emotion_loss": 0.9944021105766296, "time_take": 39.58642339706421}
{"emotion_correct": 20, "emotion_loss": 1.1266114711761475, "time_take": 39.92361402511597}
{"emotion_correct": 17, "emotion_loss": 1.4983452558517456, "time_take": 40.34286022186279}
{"emotion_correct": 19, "emotion_loss": 1.283783197402954, "time_take": 40.55108189582825}
{"emotion_correct": 16, "emotion_loss": 1.4165992736816406, "time_take": 40.92664074897766}
{"emotion_correct": 20, "emotion_loss": 1.0422054529190063, "time_take": 41.21398186683655}
{"emotion_correct": 16, "emotion_loss": 1.4197936058044434, "time_take": 41.58112025260925}
{"emotion_correct": 19, "emotion_loss": 1.1911001205444336, "time_take": 41.963061571121216}
{"emotion_correct": 16, "emotion_loss": 1.3682060241699219, "time_take": 42.30328369140625}
{"emotion_correct": 17, "emotion_loss": 1.2680983543395996, "time_take": 42.5404

{"emotion_correct": 18, "emotion_loss": 1.159940481185913, "time_take": 71.34527850151062}
{"emotion_correct": 17, "emotion_loss": 1.3110231161117554, "time_take": 71.81379723548889}
{"emotion_correct": 18, "emotion_loss": 1.0039044618606567, "time_take": 72.17203259468079}
{"emotion_correct": 19, "emotion_loss": 1.337498426437378, "time_take": 72.50746583938599}
{"emotion_correct": 23, "emotion_loss": 0.9837309718132019, "time_take": 72.89152717590332}
{"emotion_correct": 19, "emotion_loss": 1.3307442665100098, "time_take": 73.20710873603821}
{"emotion_correct": 23, "emotion_loss": 0.8474221229553223, "time_take": 73.50714564323425}
{"emotion_correct": 14, "emotion_loss": 1.5887783765792847, "time_take": 73.83044910430908}
{"emotion_correct": 19, "emotion_loss": 1.1323200464248657, "time_take": 74.39849734306335}
{"emotion_correct": 16, "emotion_loss": 1.3807227611541748, "time_take": 75.25954794883728}
{"emotion_correct": 24, "emotion_loss": 0.858055591583252, "time_take": 75.7464232

{"emotion_correct": 15, "emotion_loss": 1.3833701610565186, "time_take": 102.80195188522339}
{"emotion_correct": 15, "emotion_loss": 1.3777717351913452, "time_take": 103.1028060913086}
{"emotion_correct": 16, "emotion_loss": 1.3622740507125854, "time_take": 103.36703109741211}
{"emotion_correct": 14, "emotion_loss": 1.404280662536621, "time_take": 103.71617412567139}
{"emotion_correct": 20, "emotion_loss": 1.1728076934814453, "time_take": 104.02106809616089}
{"emotion_correct": 18, "emotion_loss": 0.9544183611869812, "time_take": 104.28848886489868}
{"emotion_correct": 17, "emotion_loss": 1.3296078443527222, "time_take": 104.75133538246155}
{"emotion_correct": 20, "emotion_loss": 1.263679027557373, "time_take": 105.1404435634613}
{"emotion_correct": 21, "emotion_loss": 1.0670866966247559, "time_take": 105.5072968006134}
{"emotion_correct": 21, "emotion_loss": 1.1928653717041016, "time_take": 105.8774561882019}
{"emotion_correct": 19, "emotion_loss": 1.3660857677459717, "time_take": 106

{"emotion_correct": 20, "emotion_loss": 1.1839697360992432, "time_take": 134.10980653762817}
{"emotion_correct": 23, "emotion_loss": 1.0586597919464111, "time_take": 134.43371510505676}
{"emotion_correct": 17, "emotion_loss": 1.1668169498443604, "time_take": 134.7236499786377}
{"emotion_correct": 17, "emotion_loss": 1.6465497016906738, "time_take": 135.09175610542297}
{"emotion_correct": 22, "emotion_loss": 0.9161148071289062, "time_take": 135.421302318573}
{"emotion_correct": 19, "emotion_loss": 1.1415895223617554, "time_take": 135.75125765800476}
{"emotion_correct": 20, "emotion_loss": 1.2179243564605713, "time_take": 136.07259464263916}
{"emotion_correct": 17, "emotion_loss": 1.5399380922317505, "time_take": 136.5212483406067}
{"emotion_correct": 18, "emotion_loss": 1.278839111328125, "time_take": 137.03240275382996}
{"emotion_correct": 17, "emotion_loss": 1.4076995849609375, "time_take": 137.3413209915161}
{"emotion_correct": 18, "emotion_loss": 1.4541505575180054, "time_take": 137

...

{"emotion_correct": 18, "emotion_loss": 1.467892050743103, "time_take": 854.0785436630249}
{"emotion_correct": 25, "emotion_loss": 0.7090092897415161, "time_take": 854.5435025691986}
{"emotion_correct": 20, "emotion_loss": 0.9617137908935547, "time_take": 854.8748998641968}
{"emotion_correct": 22, "emotion_loss": 1.1845388412475586, "time_take": 855.3473327159882}
{"emotion_correct": 19, "emotion_loss": 0.9404422640800476, "time_take": 855.6049773693085}
{"emotion_correct": 18, "emotion_loss": 1.293255090713501, "time_take": 855.8103461265564}
{"emotion_correct": 22, "emotion_loss": 1.1605477333068848, "time_take": 856.062667131424}
{"emotion_correct": 22, "emotion_loss": 1.1728050708770752, "time_take": 856.3590116500854}
{"emotion_correct": 20, "emotion_loss": 1.032222032546997, "time_take": 856.6326758861542}
{"emotion_correct": 18, "emotion_loss": 1.263533353805542, "time_take": 857.0398945808411}
{"emotion_correct": 18, "emotion_loss": 1.2191228866577148, "time_take": 857.25656223

{"emotion_correct": 26, "emotion_loss": 0.789826512336731, "time_take": 885.552375793457}
{"emotion_correct": 19, "emotion_loss": 1.1399189233779907, "time_take": 885.9694020748138}
{"emotion_correct": 22, "emotion_loss": 1.007720947265625, "time_take": 886.2692587375641}
{"emotion_correct": 17, "emotion_loss": 1.3078676462173462, "time_take": 886.6291406154633}
{"emotion_correct": 20, "emotion_loss": 1.0088369846343994, "time_take": 887.0563941001892}
{"emotion_correct": 22, "emotion_loss": 0.8711274862289429, "time_take": 887.3769834041595}
{"emotion_correct": 16, "emotion_loss": 1.3881657123565674, "time_take": 887.6173326969147}
{"emotion_correct": 20, "emotion_loss": 1.0841658115386963, "time_take": 887.9676489830017}
{"emotion_correct": 21, "emotion_loss": 1.24073326587677, "time_take": 888.4664304256439}
{"emotion_correct": 17, "emotion_loss": 1.1334922313690186, "time_take": 888.9119215011597}
{"emotion_correct": 22, "emotion_loss": 0.9954370260238647, "time_take": 889.31004261

{"emotion_correct": 18, "emotion_loss": 1.1850674152374268, "time_take": 917.7945113182068}
{"emotion_correct": 15, "emotion_loss": 1.4555420875549316, "time_take": 918.1615164279938}
{"emotion_correct": 19, "emotion_loss": 0.9902083277702332, "time_take": 918.6166052818298}
{"emotion_correct": 20, "emotion_loss": 1.249837875366211, "time_take": 919.0409288406372}
{"emotion_correct": 22, "emotion_loss": 0.9483213424682617, "time_take": 919.33274102211}
{"emotion_correct": 21, "emotion_loss": 0.9080009460449219, "time_take": 919.6817443370819}
{"emotion_correct": 17, "emotion_loss": 1.3501898050308228, "time_take": 919.9798769950867}
{"emotion_correct": 17, "emotion_loss": 1.4020166397094727, "time_take": 920.2830517292023}
{"emotion_correct": 20, "emotion_loss": 1.0170788764953613, "time_take": 920.596143245697}
{"emotion_correct": 19, "emotion_loss": 1.5104525089263916, "time_take": 920.845733165741}
{"emotion_correct": 21, "emotion_loss": 1.048097848892212, "time_take": 921.190629482

{"emotion_correct": 22, "emotion_loss": 1.069986343383789, "time_take": 950.1794369220734}
{"emotion_correct": 21, "emotion_loss": 0.9913485646247864, "time_take": 950.3694236278534}
{"emotion_correct": 19, "emotion_loss": 1.445752501487732, "time_take": 950.8214089870453}
{"emotion_correct": 15, "emotion_loss": 1.323038101196289, "time_take": 951.1923625469208}
{"emotion_correct": 24, "emotion_loss": 0.9126095175743103, "time_take": 951.5660979747772}
{"emotion_correct": 15, "emotion_loss": 1.492382526397705, "time_take": 951.7725677490234}
{"emotion_correct": 20, "emotion_loss": 1.0878381729125977, "time_take": 952.0046038627625}
{"emotion_correct": 14, "emotion_loss": 1.3889623880386353, "time_take": 952.2402474880219}
{"emotion_correct": 15, "emotion_loss": 1.3715816736221313, "time_take": 952.624915599823}
{"emotion_correct": 23, "emotion_loss": 0.8930097818374634, "time_take": 953.0911793708801}
{"emotion_correct": 20, "emotion_loss": 1.170069932937622, "time_take": 953.367928504

{"emotion_correct": 21, "emotion_loss": 1.063156247138977, "time_take": 980.4507150650024}
{"emotion_correct": 16, "emotion_loss": 1.35820472240448, "time_take": 980.8080666065216}
{"emotion_correct": 19, "emotion_loss": 1.1448736190795898, "time_take": 981.0952923297882}
{"emotion_correct": 23, "emotion_loss": 1.045968770980835, "time_take": 981.6012794971466}
{"emotion_correct": 21, "emotion_loss": 1.131397008895874, "time_take": 981.9693126678467}
{"emotion_correct": 18, "emotion_loss": 1.0630900859832764, "time_take": 982.2379939556122}
{"emotion_correct": 12, "emotion_loss": 1.789674997329712, "time_take": 982.5642039775848}
{"emotion_correct": 22, "emotion_loss": 0.9932622313499451, "time_take": 982.8989226818085}
{"emotion_correct": 18, "emotion_loss": 1.3627687692642212, "time_take": 983.448816537857}
{"emotion_correct": 18, "emotion_loss": 1.385454535484314, "time_take": 983.6670567989349}
{"emotion_correct": 18, "emotion_loss": 1.1397730112075806, "time_take": 983.94647550582